<a href="https://colab.research.google.com/github/capture120/sp2025ir/blob/main/retrieval-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Models and Evaluation



In this notebook, you will evaluate results ranking on a test collection. First, you'll compute the mean average precision of a baseline BM25 model. Then you'll implement a query-likelihood retrieval model based on the same inverted index.

This notebook uses the [Pyserini](http://pyserini.io/) library, a Python interface to [Anserini](http://anserini.io) and thus to [Lucene](https://lucene.apache.org/), a widely-used open-source search engine. This library is written and maintained by Jimmy Lin and his colleagues at the University of Waterloo.


We start by installing the python interface. Since it calls the underlying Lucene search engine, which is written in Java, we make sure we point to an appropriate Java installation. If like Colab you don't have Java 21, uncomment the following code and run it, or whatever makes sense for your platform.

In [ ]:
## Uncomment the following code to install Java 21 on Colab
#!apt-get install openjdk-21-jre-headless -qq > /dev/null
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
#!update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
#!java -version

openjdk version "21.0.6" 2025-01-21
OpenJDK Runtime Environment (build 21.0.6+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 21.0.6+7-Ubuntu-122.04.1, mixed mode, sharing)


In [20]:
!pip install pyserini
# You can change this to gpu if you have one.
# It's a pyserini dependency, but we won't need it until the next assignment.
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 26.2 MB/s eta 0:00:00


You can use the `LuceneSearcher` to search over an index. We can initialize the searcher with a pre-built index, which Pyserini will automatically download if it hasn't already. Note that the index takes up 1.6GB of disk.

In [8]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('robust04')

Now we can search for a query and inspect the results:

In [9]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [10]:
from pyserini.index import LuceneIndexReader
from IPython.core.display import display, HTML

reader = LuceneIndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [11]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency).
For brevity, we only look at terms that appear more than once:

In [12]:
doc_vector = reader.get_document_vector('LA092790-0015')
{ k: v for (k, v) in doc_vector.items() if v >1 }

{'been': 11,
 'habitat': 2,
 'year': 3,
 'jenk': 4,
 'mountain': 3,
 'rees': 3,
 'would': 2,
 'debusscher': 3,
 'near': 3,
 'hungri': 3,
 'past': 2,
 'especi': 2,
 'wildlif': 2,
 'lion': 3,
 'ventura': 4,
 'anim': 9,
 'forest': 2,
 'two': 3,
 'seen': 3,
 'attack': 3,
 'black': 2,
 'i': 2,
 'food': 3,
 'counti': 4,
 'rural': 3,
 "they'r": 5,
 'sever': 3,
 'california': 2,
 'advis': 2,
 'parch': 2,
 'area': 4,
 'she': 3,
 'month': 4,
 'deer': 3,
 'famili': 2,
 'we': 2,
 'peopl': 2,
 'just': 3,
 'live': 2,
 'drought': 4,
 'ha': 3,
 'he': 2,
 'hi': 2,
 'septemb': 2,
 'yard': 2,
 'elsewher': 2,
 'natur': 2,
 'out': 4,
 'lot': 2,
 'have': 16,
 'leav': 2,
 'more': 3,
 'off': 2,
 'ojai': 3,
 'report': 4,
 'bobcat': 4,
 'coyot': 10,
 'few': 2,
 'bear': 4,
 'depart': 2,
 'author': 3,
 'dry': 2,
 'vallei': 4,
 'who': 2,
 'game': 2,
 'resid': 2,
 'hous': 3,
 'cat': 3,
 'sai': 2,
 'said': 19,
 'park': 2,
 'on': 2,
 'offici': 5,
 'wild': 3}

## Evaluating Ranked Results

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the TREC conferences call them.

In [13]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: TREC's term for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [14]:
topics[301]

{'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [15]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

Now, we'll fetch the relevance judgments for the Robust04 queries, which TREC calls "qrels".

In [16]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
#qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [17]:
qrels[0:10]

[[301, 0, 'FBIS3-10082', 1],
 [301, 0, 'FBIS3-10169', 0],
 [301, 0, 'FBIS3-10243', 1],
 [301, 0, 'FBIS3-10319', 0],
 [301, 0, 'FBIS3-10397', 1],
 [301, 0, 'FBIS3-10491', 1],
 [301, 0, 'FBIS3-10555', 0],
 [301, 0, 'FBIS3-10622', 1],
 [301, 0, 'FBIS3-10634', 0],
 [301, 0, 'FBIS3-10635', 0]]

## Computing Mean Average Precision

The Robust04 collection uses binary relevance judgments and usually has multiple relevant results for each query. It is thus common to use **mean average precision** (MAP) to evaluate retrieval performance on it. Remember from class that MAP adds the precision at the position of each _relevant_ document in a ranked list and then divides by the total number of relevant documents. So that we don't have to scan through the entire collection, we usually evaluate MAP at some maximum rank value, such as 100 or 1000. We simply stop scanning at that maximum rank.

As we saw above, you should pass a query string (the `title` of a topic) and the desired number of results to the `search` method of the `searcher` object.

In [18]:
hits = searcher.search(dev_topics[355]['title'], 1000)
[(hit.docid, hit.score) for hit in hits[0:10]]

[('FBIS4-20436', 11.349300384521484),
 ('FBIS3-23683', 9.126500129699707),
 ('FBIS3-21238', 8.309300422668457),
 ('FBIS4-44915', 7.935699939727783),
 ('FBIS4-20602', 7.6006999015808105),
 ('FBIS4-47382', 7.529600143432617),
 ('FT943-1589', 7.480100154876709),
 ('LA071789-0059', 7.451399803161621),
 ('FBIS4-22145', 7.215099811553955),
 ('FBIS4-44667', 7.106500148773193)]

For this assignment, evaluate MAP@1000 for the list of `test_topics` we created above. You should process the `qrels` data to find the relevant results for each query.

In [22]:
## TODO [35 points]: Compute MAP@1000 for test_topics

def calculate_map_at_1000(searcher, test_topics, qrels):
    qrels_by_topic = {}
    for item in qrels:
        topic_id = item[0]
        if topic_id not in qrels_by_topic:
            qrels_by_topic[topic_id] = []
        qrels_by_topic[topic_id].append(item)
    ap_values = []
    processed_topics = 0

    for topic_id, topic_data in test_topics.items():
        processed_topics += 1

        query = topic_data['title']

        if topic_id not in qrels_by_topic:
            continue

        relevant_docs = set([item[2] for item in qrels_by_topic[topic_id] if item[3] == 1])

        if not relevant_docs:
            continue

        hits = searcher.search(query, 1000)

        seen_relevant = 0
        precision_sum = 0.0

        for i, hit in enumerate(hits):
            if i >= 1000:
                break

            if hit.docid in relevant_docs:
                seen_relevant += 1
                precision_at_k = seen_relevant / (i + 1)
                precision_sum += precision_at_k

        ap = precision_sum / len(relevant_docs)
        ap_values.append(ap)

    map_at_1000 = sum(ap_values) / len(ap_values) if ap_values else 0.0
    return map_at_1000

searcher = LuceneSearcher.from_prebuilt_index('robust04')

map_score = calculate_map_at_1000(searcher, test_topics, qrels)
print(f"MAP@1000: {map_score}")

MAP@1000: 0.23485221629843145


## Implementing a Query Likelihood Model

The default `LuceneSearcher` in pyserini uses a BM25 model. In this part of the assignment, you'll interact directly with the index to implement a query likelihood retrieval model.  For more details on pyserini's index API, [see the documentation](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md).

We've already created an `IndexReader` for the Robust04 collection and assigned it to `reader`. Remember that we can pass documents or queries through the index's predefined analysis pipeline, a series of operations like stemming, stopword removal, etc.

In [24]:
reader.analyze('Are there black bear attacks there?')

['black', 'bear', 'attack']

For most retrieval models, including BM25 and query likelihood, we'll want to know the overall statistics for the terms we retrieve from the index. By default, pyserini will do stemming and other "analysis" steps on the terms you give it. But since we're already working with stemmed queries, we'll tell it not to perform this analysis now.

In [25]:
# Just checking that our term is already in its index form.
print(reader.analyze('bear'))

df, cf = reader.get_term_counts('bear', analyzer=None)
print(f'The document frequency is {df} and the collection frequency is {cf}.')

['bear']
The document frequency is 16429 and the collection frequency is 23545.


You might also want to know summary statistics about the collection.

In [26]:
reader.stats()

{'total_terms': 174540872,
 'documents': 528030,
 'non_empty_documents': 528030,
 'unique_terms': 923436}

**Now** let's traverse the postings for a term.

In [30]:
posting_list = reader.get_postings_list('bear', analyzer=None)
print(f'There are {len(posting_list)} postings.')

posting = posting_list[100]
print(f'Each posting contains a document ID ({posting.docid}), term frequency ({posting.tf}), and position list ({posting.positions}).')

print(f'searcher has a method for turning internal integer IDs like {posting.docid} into external IDs like {searcher.doc(posting.docid).docid()}.')

There are 16429 postings.
Each posting contains a document ID (3405), term frequency (1), and position list ([461]).
searcher has a method for turning internal integer IDs like 3405 into external IDs like LA031889-0009.


If, instead of mapping terms to documents, you want to use a document name to get term information, see above for the `get_document_vector` method.

Finally, it will be useful to have some information about each document in the collection, which is saved in a file in the GitHub repository.

In [31]:
import json
docfile = urlopen('https://github.com/dasmiq/cs6200-retrieval-models/raw/refs/heads/main/doclen.jsonl')
docinfo = [json.loads(line) for line in docfile]

Each record in that file contains the document `id`, its interned numeric `iid`, and the number of tokens `n`.

In [32]:
docinfo[0]

{'id': 'LA052189-0001', 'iid': 0, 'n': 6}

Use Dirichlet smoothing with $\mu$ equal to the mean document length in the collection.

In [34]:
 # TODO [2 points]: Compute the mean document length from collection statistics.
total_tokens = sum(doc['n'] for doc in docinfo)
total_docs = len(docinfo)
mu = total_tokens / total_docs
mu

330.5510520235593

Now you should have all the information you need to implement a query likelihood retrieval model.

In [35]:
## TODO [50 points]: Implement a query likelihood model with Dirichlet smoothing.
## For a given query, index reader, and constant k, return the k documents with the highest query likelihoods.
## Think about what data structure would let you keep the top k.
import heapq
import math
from collections import defaultdict

def query_likelihood_retrieval(query, reader, searcher, k, mu, docinfo):
    query_terms = reader.analyze(query)
    print(f"Query terms after analysis: {query_terms}")
    total_tokens = sum(doc['n'] for doc in docinfo)

    docid_to_length = {doc['id']: doc['n'] for doc in docinfo}
    doc_scores = defaultdict(float)

    internal_to_external = {}
    doc_lengths = {}

    for term in query_terms:
        try:
            df, cf = reader.get_term_counts(term, analyzer=None)

            if df == 0:
                print(f"Term '{term}' not found in collection")
                continue

            print(f"Term '{term}': df={df}, cf={cf}")

            p_t_C = cf / total_tokens

            posting_list = reader.get_postings_list(term, analyzer=None)

            for posting in posting_list:
                internal_docid = posting.docid
                tf = posting.tf

                try:
                    if internal_docid in internal_to_external:
                        external_docid = internal_to_external[internal_docid]
                    else:
                        external_docid = searcher.doc(internal_docid).docid()
                        internal_to_external[internal_docid] = external_docid

                    if external_docid in doc_lengths:
                        doc_length = doc_lengths[external_docid]
                    else:
                        doc_length = docid_to_length.get(external_docid, 0)
                        doc_lengths[external_docid] = doc_length

                    if doc_length > 0:
                        p_t_d = (tf + mu * p_t_C) / (doc_length + mu)
                        log_p_t_d = math.log(p_t_d)

                        doc_scores[external_docid] += log_p_t_d
                except:
                    continue
        except:
            print(f"Error processing term '{term}'")
            continue

    top_k_docs = heapq.nlargest(k, doc_scores.items(), key=lambda x: x[1])

    print(f"Retrieved {len(top_k_docs)} documents")
    if len(top_k_docs) > 0:
        print(f"Top document: {top_k_docs[0][0]} with score {top_k_docs[0][1]:.4f}")

    return top_k_docs

hits = query_likelihood_retrieval("international organized crime", reader, searcher, 1000, mu, docinfo)

Query terms after analysis: ['intern', 'organ', 'crime']
Term 'intern': df=114475, cf=197933
Term 'organ': df=51239, cf=115735
Term 'crime': df=14955, cf=30996
Retrieved 1000 documents
Top document: LA100790-0072 with score -3.1297


Now you can reuse some of your evaluation code for mean average precision from above.

In [ ]:
## TODO [13 points]: Compute the MAP@1000 of your query likelihood model on the test_topics set.

In [37]:
def calculate_map_at_1000(retrieval_function, reader, searcher, test_topics, qrels, mu, docinfo):
    qrels_by_topic = {}
    for item in qrels:
        topic_id = item[0]
        if topic_id not in qrels_by_topic:
            qrels_by_topic[topic_id] = []
        qrels_by_topic[topic_id].append(item)

    ap_values = []
    processed_topics = 0

    for topic_id, topic_data in test_topics.items():
        processed_topics += 1
        print(f"Processing topic {topic_id}: {topic_data['title']}")

        if topic_id not in qrels_by_topic:
            print(f"No relevance judgments for topic {topic_id}")
            continue

        query = topic_data['title']

        relevant_docs = set([item[2] for item in qrels_by_topic[topic_id] if item[3] == 1])

        if not relevant_docs:
            print(f"No relevant documents for topic {topic_id}")
            continue

        print(f"Found {len(relevant_docs)} relevant documents for topic {topic_id}")

        hits = retrieval_function(query, reader, searcher, 1000, mu, docinfo)

        seen_relevant = 0
        precision_sum = 0.0

        for i, (docid, _) in enumerate(hits):
            if docid in relevant_docs:
                seen_relevant += 1
                precision_at_k = seen_relevant / (i + 1)
                precision_sum += precision_at_k

        ap = precision_sum / len(relevant_docs)
        ap_values.append(ap)

        print(f"AP@1000 for topic {topic_id}: {ap:.4f} ({seen_relevant}/{len(relevant_docs)} relevant docs found)")

    map_at_1000 = sum(ap_values) / len(ap_values) if ap_values else 0.0

    print(f"\nProcessed {processed_topics} topics, calculated AP for {len(ap_values)} topics")
    print(f"MAP@1000 for Query Likelihood model: {map_at_1000:.4f}")

    return map_at_1000

def evaluate_query_likelihood_model(reader, searcher, test_topics, qrels, docinfo):
    total_tokens = sum(doc['n'] for doc in docinfo)
    total_docs = len(docinfo)
    mu = total_tokens / total_docs
    print(f"Dirichlet smoothing parameter μ (mean document length): {mu:.2f}")

    map_score = calculate_map_at_1000(
        query_likelihood_retrieval,
        reader,
        searcher,
        test_topics,
        qrels,
        mu,
        docinfo
    )

    return map_score

map_score = evaluate_query_likelihood_model(reader, searcher, test_topics, qrels, docinfo)

Dirichlet smoothing parameter μ (mean document length): 330.55
Processing topic 658: teenage pregnancy
Found 37 relevant documents for topic 658
Query terms after analysis: ['teenag', 'pregnanc']
Term 'teenag': df=1049, cf=1287
Term 'pregnanc': df=1096, cf=2008
Retrieved 1000 documents
Top document: FBIS3-59301 with score -3.6638
AP@1000 for topic 658: 0.0127 (18/37 relevant docs found)
Processing topic 417: creativity
Found 75 relevant documents for topic 417
Query terms after analysis: ['creativ']
Term 'creativ': df=4955, cf=7073
Retrieved 1000 documents
Top document: LA092989-0098 with score -3.5800
AP@1000 for topic 417: 0.2863 (58/75 relevant docs found)
Processing topic 659: cruise health safety
Found 13 relevant documents for topic 659
Query terms after analysis: ['cruis', 'health', 'safeti']
Term 'cruis': df=2518, cf=5020
Term 'health': df=31528, cf=74088
Term 'safeti': df=20221, cf=48043
Retrieved 1000 documents
Top document: FT922-14532 with score -2.7562
AP@1000 for topic 65

In [39]:
print(f"Map@1000 {map_score}")

Map@1000 0.03061478033718738
